# Market Analysis

In [ ]:
import sys
sys.path.append("..")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.intersection import intersection
from matplotlib import rcParams 
from sklearn.metrics import roc_curve

In [ ]:
rcParams.update({'font.size': 12})
rcParams.update({'figure.autolayout': True})

In [ ]:
def supCurve(df):
    curve = pd.DataFrame(columns=['OPS','P','Q'])
    cnt = 0
    for op in df['OPERATORE'].unique():
        new = pd.DataFrame(columns=['OPS','P','Q'])
        temp = df.where(df['OPERATORE']==op).dropna()
        new.loc[cnt] = [
            op,
            np.mean(temp['ENERGY_PRICE_NO']),
            np.sum(temp['QUANTITY_NO'])
        ]
        cnt+=1
        curve = pd.concat([curve, new], axis= 0)
    
    curve = curve.sort_values('P', ascending=True)
    
    return curve

In [ ]:
def demCurve(df):
    df['ENERGY_PRICE_NO'] = df['ENERGY_PRICE_NO'].replace(0, 3000)
    curve = pd.DataFrame(columns=['OPS','P','Q'])
    cnt = 0
    for op in df['OPERATORE'].unique():
        new = pd.DataFrame(columns=['OPS','P','Q'])
        temp = df.where(df['OPERATORE']==op).dropna()
        new.loc[cnt] = [
            op,
            np.mean(temp['ENERGY_PRICE_NO']),
            np.sum(temp['QUANTITY_NO'])
        ]
        cnt+=1
        curve = pd.concat([curve, new], axis= 0)
    
    curve = curve.sort_values('P', ascending=False)
    
    return curve

## Bilateral Analysis
### Demand/Supply Curves: No Bilateral

In [ ]:
data = (
    pd
    .read_csv('../data/data.csv')
    .drop(columns=[
        'MARKET_CD', 
        'UNIT_REFERENCE_NO', 
        'MARKET_PARTECIPANT_XREF_NO',
        'BID_OFFER_DATE_DT',
        'TRANSACTION_REFERENCE_NO',
        'MERIT_ORDER_NO',
        'PARTIAL_QTY_ACCEPTED_IN',
        'ADJ_QUANTITY_NO',
        'ADJ_ENERGY_PRICE_NO',
        'GRID_SUPPLY_POINT_NO',
        'SUBMITTED_DT',
        'BALANCED_REFERENCE_NO',
    ])
)

In [ ]:
acc = (
    data
    .where(data['OPERATORE']!='Bilateralista')
    .where(data['STATUS_CD'].isin(['ACC', 'REJ']))
    .dropna()
)
off = acc.where(acc['PURPOSE_CD']=='OFF').dropna().drop(columns='PURPOSE_CD')
bid = acc.where(acc['PURPOSE_CD']=='BID').dropna().drop(columns='PURPOSE_CD')
sup = supCurve(off)
dem = demCurve(bid)

Cumulative sums of quantity

In [ ]:
sup_cum = np.cumsum(sup['Q'])
dem_cum = np.cumsum(dem['Q'])

Determine the clearing price as the curves intersection point

In [ ]:
clearing1 = intersection(sup_cum.values, sup.P.values, dem_cum.values, dem.P.values)[1][0]

In [ ]:
plt.figure()
plt.plot(sup_cum, sup.P, label='Supply', linewidth=2)
plt.plot(dem_cum, dem.P, label='Demand', linewidth=2)
plt.axhline(y=clearing1, linestyle='-.', color='k')
plt.ylim(0, 500)
plt.grid(linestyle='-.')
plt.legend()

plt.xlabel('Quantity [MWh]')
plt.ylabel('Price [\u20ac/MWh]')
plt.savefig('../fig/CurvesNoB.png', transparent=True)

Get the True Positive Rate and the False Positive Rate which will be used in the ROC curves

In [ ]:
obs_off = off['STATUS_CD'].replace('ACC', 1).replace('REJ',0)
obs_bid = bid['STATUS_CD'].replace('ACC', 1).replace('REJ',0)
pred_off = off.where(off['ENERGY_PRICE_NO']>clearing1)['STATUS_CD'].replace(['ACC', 'REJ'],0).fillna(1)
pred_bid = bid.where(bid['ENERGY_PRICE_NO']<clearing1)['STATUS_CD'].replace(['ACC', 'REJ'],0).fillna(1)
obs = pd.concat([obs_off, obs_bid], axis= 0)
pred = pd.concat([pred_off, pred_bid], axis= 0)

fpr, tpr, thresholds = roc_curve(obs, pred)

### Demand/Supply Curves: No Bilateral

In [ ]:
data = (
    pd
    .read_csv('../data/data.csv')
    .drop(columns=[
        'MARKET_CD', 
        'UNIT_REFERENCE_NO', 
        'MARKET_PARTECIPANT_XREF_NO',
        'BID_OFFER_DATE_DT',
        'TRANSACTION_REFERENCE_NO',
        'MERIT_ORDER_NO',
        'PARTIAL_QTY_ACCEPTED_IN',
        'ADJ_QUANTITY_NO',
        'ADJ_ENERGY_PRICE_NO',
        'GRID_SUPPLY_POINT_NO',
        'SUBMITTED_DT',
        'BALANCED_REFERENCE_NO',
    ])
)

In [ ]:
acc = (
    data
    .where(data['STATUS_CD'].isin(['ACC', 'REJ']))
    .dropna()
)
off = acc.where(acc['PURPOSE_CD']=='OFF').dropna().drop(columns='PURPOSE_CD')
bid = acc.where(acc['PURPOSE_CD']=='BID').dropna().drop(columns='PURPOSE_CD')
sup = supCurve(off)
dem = demCurve(bid)

Cumulative sums of quantity

In [ ]:
sup_cum = np.cumsum(sup['Q'])
dem_cum = np.cumsum(dem['Q'])

Determine the clearing price as the curves intersection point

In [ ]:
clearing2 = intersection(sup_cum.values, sup.P.values, dem_cum.values, dem.P.values)[1][0]

In [ ]:
plt.figure()
plt.plot(sup_cum, sup.P, label='Supply', linewidth=2)
plt.plot(dem_cum, dem.P, label='Demand', linewidth=2)
plt.axhline(y=clearing2, linestyle='-.', color='k')
plt.ylim(0, 500)
plt.grid(linestyle='-.')

plt.legend()
plt.xlabel('Quantity [MWh]')
plt.ylabel('Price [\u20ac/MWh]')
plt.savefig('../fig/CurvesB.png', transparent=True)

Get the True Positive Rate and the False Positive Rate which will be used in the ROC curves

In [ ]:
obs_off = off['STATUS_CD'].replace('ACC', 1).replace('REJ',0)
obs_bid = bid['STATUS_CD'].replace('ACC', 1).replace('REJ',0)
pred_off = off.where(off['ENERGY_PRICE_NO']>clearing2)['STATUS_CD'].replace(['ACC', 'REJ'],0).fillna(1)
pred_bid = bid.where(bid['ENERGY_PRICE_NO']<clearing2)['STATUS_CD'].replace(['ACC', 'REJ'],0).fillna(1)
obs = pd.concat([obs_off, obs_bid], axis= 0)
pred = pd.concat([pred_off, pred_bid], axis= 0)

fprB, tprB, thresholds = roc_curve(obs, pred)

### ROC Curves

In [ ]:
plt.figure()
plt.plot(fpr, tpr, label='NoBilateral', linewidth=2)
plt.plot(fprB, tprB, label='AggBilateral', linestyle='-.', linewidth=2)
plt.plot([0, 1], [0, 1], color='k', linestyle=':')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.savefig('../fig/roc.png', transparent=True)

# Market Coupling

In [ ]:
from influxdb import InfluxDBClient
import pandas as pd
import sys
sys.path.append("..")
import matplotlib.pyplot as plt
from src.intersection import intersection
import numpy as np
from matplotlib import rcParams
rcParams.update({'font.size': 12})
rcParams.update({'figure.autolayout': True})

## MGP Coupling

In [ ]:
client = InfluxDBClient('localhost', 8086, 'root', 'root', 'PublicBids')
res = client.query("select * from demandMGP where time='2020-04-03T00:00:00Z'").raw
bid = pd.DataFrame(res['series'][0]['values'], columns = res['series'][0]['columns']).set_index('op').drop(columns=['time'])
res = client.query("select * from supplyMGP where time='2020-04-03T00:00:00Z'").raw
off = pd.DataFrame(res['series'][0]['values'], columns = res['series'][0]['columns']).set_index('op').drop(columns=['time'])
bid.Price = bid.Price.replace(0, 3000)

In [ ]:
sup = off[(off >= 0).any(1)]
dem = bid[(bid >= 0).any(1)]
# Sort the prices
sup = off.sort_values('Price', ascending=True)
dem = bid.sort_values('Price', ascending=False)
# Cumulative sums of quantity
sup_cum = np.cumsum(sup['Quantity'])
dem_cum = np.cumsum(dem['Quantity'])
clearing = intersection( 
    dem_cum.values, 
    dem.Price.values, 
    sup_cum.values, 
    sup.Price.values,
)

In [ ]:
plt.figure(figsize=(9,8))
plt.plot(dem_cum, dem.Price.values, linewidth = 2, label='Demand')
plt.plot(sup_cum, sup.Price.values, linewidth = 2, label='Supply')
plt.axhline(y=clearing[1][0], linestyle='-.', color='k')
plt.ylim(0, 300)
plt.grid(linestyle='-.')
plt.legend()

plt.xlabel('Quantity [MWh]')
plt.ylabel('Price [\u20ac/MWh]')
plt.savefig('../fig/CurvesMGP.png', transparent=True)
plt.close()

## MI Coupling

In [ ]:
client = InfluxDBClient('localhost', 8086, 'root', 'root', 'PublicBids')
res = client.query("select * from demandMI where time='2020-04-03T00:00:00Z'").raw
bid = pd.DataFrame(res['series'][0]['values'], columns = res['series'][0]['columns']).set_index('op').drop(columns=['time'])
res = client.query("select * from supplyMI where time='2020-04-03T00:00:00Z'").raw
off = pd.DataFrame(res['series'][0]['values'], columns = res['series'][0]['columns']).set_index('op').drop(columns=['time'])
bid.Price = bid.Price.replace(0, 3000)

In [ ]:
sup = off[(off >= 0).any(1)]
dem = bid[(bid >= 0).any(1)]
# Sort the prices
sup = off.sort_values('Price', ascending=True)
dem = bid.sort_values('Price', ascending=False)
# Cumulative sums of quantity
sup_cum = np.cumsum(sup['Quantity'])
dem_cum = np.cumsum(dem['Quantity'])
clearing = intersection( 
    dem_cum.values, 
    dem.Price.values, 
    sup_cum.values, 
    sup.Price.values,
)

In [ ]:
plt.figure(figsize=(9,8))
plt.plot(dem_cum, dem.Price.values, linewidth = 2, label='Supply')
plt.plot(sup_cum, sup.Price.values, linewidth = 2, label='Demand')
plt.axhline(y=clearing[1][0], linestyle='-.', color='k')
plt.ylim(0, 300)
plt.grid(linestyle='-.')
plt.legend()

plt.xlabel('Quantity [MWh]')
plt.ylabel('Price [\u20ac/MWh]')
plt.savefig('../fig/CurvesMI.png', transparent=True)
plt.close()

## MSD Coupling

In [ ]:
th = client.query(f"SELECT * FROM STRes where time='2020-04-03T00:00:00Z'").raw['series'][0]['values'][0][1]

In [ ]:
client = InfluxDBClient('localhost', 8086, 'root', 'root', 'PublicBids')
res = client.query("select * from demandMSD where time='2020-04-03T00:00:00Z'").raw
bid = pd.DataFrame(res['series'][0]['values'], columns = res['series'][0]['columns']).set_index('op').drop(columns=['time'])
res = client.query("select * from supplyMSD where time='2020-04-03T00:00:00Z'").raw
off = pd.DataFrame(res['series'][0]['values'], columns = res['series'][0]['columns']).set_index('op').drop(columns=['time'])
bid.Price = bid.Price.replace(0, 3000)

In [ ]:
sup = off[(off >= 0).any(1)]
dem = bid[(bid >= 0).any(1)]
# Sort the prices
sup = off.sort_values('Price', ascending=True)
dem = bid.sort_values('Price', ascending=False)
# Cumulative sums of quantity
sup_cum = np.cumsum(sup['Quantity'])
dem_cum = np.cumsum(dem['Quantity'])
# Create the th curve
x_th = np.array([th, th])
y_th = np.array([0, np.max(sup.Price.values)])
clearing = intersection( 
    dem_cum.values, 
    dem.Price.values, 
    x_th,
    y_th
)

In [ ]:
plt.figure(figsize=(9,8))
plt.plot(dem_cum, dem.Price.values, linewidth = 2, label='Demand')
plt.plot(x_th, y_th, linewidth = 2, linestyle=':', color='r', label='Terna Threshold')
plt.axhline(y=clearing[1][0], linestyle='-.', color='k')
plt.ylim(0, 300)
plt.grid(linestyle='-.')
plt.legend()

plt.xlabel('Quantity [MWh]')
plt.ylabel('Price [\u20ac/MWh]')
plt.savefig('../fig/CurvesMSD_d.png', transparent=True)
plt.close()

In [ ]:
sup = off[(off >= 0).any(1)]
dem = bid[(bid >= 0).any(1)]
# Sort the prices
sup = off.sort_values('Price', ascending=True)
dem = bid.sort_values('Price', ascending=False)
# Cumulative sums of quantity
sup_cum = np.cumsum(sup['Quantity'])
dem_cum = np.cumsum(dem['Quantity'])
# Create the th curve
x_th = np.array([th, th])
y_th = np.array([0, np.max(sup.Price.values)])
clearing = intersection( 
    sup_cum.values, 
    sup.Price.values, 
    x_th,
    y_th
)

In [ ]:
plt.figure(figsize=(9,8))
plt.plot(sup_cum, sup.Price.values, linewidth = 2, label='Supply')
plt.plot(x_th, y_th, linewidth = 2, linestyle=':', color='r', label='Terna Threshold')
plt.axhline(y=clearing[1][0], linestyle='-.', color='k')
plt.ylim(0, 300)
plt.grid(linestyle='-.')
plt.legend()

plt.xlabel('Quantity [MWh]')
plt.ylabel('Price [\u20ac/MWh]')
plt.savefig('../fig/CurvesMSD_s.png', transparent=True)
plt.close()